# Multiprecision Arithmetic UXArray
In this notebook we will explore the use of the UXArray which supports multiprecision arithmetic.

In [ ]:
import numpy as np
import uxarray as ux
import gmpy2
from gmpy2 import mpfr, mpz

# Using GMPY2.mpfr type for Multiprecision Arithmetic
The distance between any two nodes is less than 1e-17 (the precision of the nodes is 60 bits, which is larger than the the python `float` type precision of 53 bits)

In [ ]:
# Generate the nodes on the unit sphere, and the nodes are extremely close to each other: the edge length
# between any two nodes is less than 1e-17 (the precision of the nodes is 60 bits but we will use 65 bits for
# better demonstration)
nodes = []
for i in range(4):
    theta = gmpy2.div(i * 2 * gmpy2.const_pi(precision=65),
                      4)  # Angle in radians
    x = gmpy2.cos(theta)
    y = gmpy2.sin(theta)
    z = mpfr(
        '0')  # All nodes will have z-coordinate as 0 on the unit sphere
    nodes.append([x, y, z])

# Generate the face nodes connectivity
dumb_nodes = [
    ux.INT_FILL_VALUE_MPZ, ux.INT_FILL_VALUE_MPZ, ux.INT_FILL_VALUE_MPZ
]

face_nodes_connectivity = np.array([
    np.array([nodes[0], nodes[1], nodes[2], dumb_nodes], dtype=object),
    np.array([nodes[0], nodes[2], nodes[3], dumb_nodes], dtype=object),
    np.array([nodes[0], nodes[3], nodes[1], dumb_nodes], dtype=object),
    np.array([nodes[1], nodes[2], nodes[3], nodes[0]], dtype=object)
], dtype=object)

Now we can see this topology has 4 faces and 4 nodes. Three of the facess have 3 nodes and one face has 4 nodes. So we also use the
fill value `ux.INT_FILL_VALUE_MPZ` to fill the fourth node of the first three faces.

In [ ]:
face_nodes_connectivity

## Construct the Grid object with the multiprecision mode on

In [ ]:
# Create the grid
vgrid = ux.Grid(face_nodes_connectivity,
                multi_precision=True,
                precision=65,
                vertices=True,
                islatlon=False,
                concave=False)

## Check the Grid Information
We can see that even though the input nodes are extremely close to each other, the grid is still valid and not degenerated:
The face number is still 4 and the node number is still 4.

In [ ]:
vgrid.nMesh2_face
vgrid.nMesh2_node

#  Using python string type for Multiprecision Arithmetic
You can also input the coordinates as string. The string will be converted to the mpfr type.

In [ ]:
# Provide nodes that are extremely closed to each other
face_nodes_connectivity = [['1.000000000000000000', '1.000000000000000000001', '1.0000000000000000000012','INT_FILL_VALUE'],
 ['1.000000000000000001', '1.000000000000000000002', '1.0000000000000000000013','1.0000000000000000000014']]

uxgrid =   ux.Grid(face_nodes_connectivity, multi_precision=True, precision= 100,
                        vertices=True,
                        islatlon=False,
                        concave=False)

## Check the Grid Information

In [ ]:
print(uxgrid.nMesh2_face)
print(uxgrid.nMesh2_node)

# Precise Conversion between Spherical and Cartesian Coordinates
By turning on the multiprecision mode, the conversion between spherical and cartesian coordinates will be exact in the given precision.

## Test for the longitude and latitude conversion helper functions with multiprecision
The following example shows that the conversion between longitude and latitude and xyz is exact in the given precision when using the
gmpy2.mpfr type. Such exact compuation is also supported in our Grid object.


In [ ]:
import uxarray as ux
import uxarray.helpers as helpers
import uxarray.multi_precision_helpers as mp_helpers


# With the multiprecision mode on, the conversion between spherical and cartesian coordinates will be exact in the given precision
# Multiprecision test for places=19, And we set the global precision places to 20
# Repeat the conversion between latitude and longitude and xyz for 1000 times
# And see if the results are the same as the initial coordinates

precision = mp_helpers.decimal_digits_to_precision_bits(20)

# Set the global precision of the mpfr numbers.
# Important Note:
# 1. To avoid arithmetic overflow, the global precision should always be higher than any other precision speicified
# in the code.
# 2. Modifying the precision by calling this function will modify all following codes running context until
# another call to this function.
mp_helpers.set_global_precision(precision)

# The initial coordinates
[init_x, init_y, init_z] = ux.helpers.normalize_in_place([mpfr('0.12345678910111213149'),mpfr('0.92345678910111213149'),mpfr('1.72345678910111213149')])
new_x = init_x
new_y = init_y
new_z = init_z
for iter in range(1000):
    [new_lon, new_lat] = ux.helpers.node_xyz_to_lonlat_rad([new_x, new_y, new_z])
    [new_x, new_y, new_z] = ux.helpers.node_lonlat_rad_to_xyz([new_lon, new_lat])
    assert new_x == init_x
    assert new_y == init_y

# Test for the longitude and latitude conversion
# The initial coordinates
[init_lon, init_lat] = [mpfr('1.4000332309896247'),mpfr('1.190289949682531')]
# Reset global precision to default
new_lat = init_lat
new_lon = init_lon
for iter in range(1000):
    [new_x, new_y, new_z] = ux.helpers.node_lonlat_rad_to_xyz([new_lon, new_lat])
    [new_lon, new_lat] = ux.helpers.node_xyz_to_lonlat_rad([new_x, new_y, new_z])
    assert new_lon == init_lon
    assert new_lat == init_lat

# Reset global precision to default
mp_helpers.set_global_precision()